In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys

In [5]:
class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = tf.keras.optimizers.RMSprop(lr=0.00005)
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss, optimizer=optimizer,metrics=['accuracy'])
        self.generator = self.build_generator()
        z = tf.keras.layers.Input(shape =(self.latent_dim,))
        img = self.generator(z)
        self.critic.trainable = False
        valid = self.critic(img)
        self.combined = tf.keras.models.Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
    
    def wasserstein_loss(self, y_true, y_pred):
        return tf.keras.backend.mean(y_true * y_pred)
    
    def build_generator(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(128 * 7 * 7, activation='relu', input_dim = self.latent_dim))
        model.add(tf.keras.layers.Reshape((7, 7, 128)))
        model.add(tf.keras.layers.UpSampling2D())
        model.add(tf.keras.layers.Conv2D(128, kernel_size=4, padding="same"))
        model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.UpSampling2D())
        model.add(tf.keras.layers.Conv2D(64, kernel_size=4, padding='same'))
        model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Conv2D(self.channels, kernel_size = 4, padding='same'))
        model.add(tf.keras.layers.Activation('tanh'))
        noise = tf.keras.layers.Input(shape=(self.latent_dim,))
        img = model(noise)
        return(tf.keras.models.Model(noise, img))
    
    def build_critic(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(tf.keras.layers.ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(tf.keras.layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1))

        
        img = tf.keras.layers.Input(shape = self.img_shape)
        validity = model(img)
        return(tf.keras.models.Model(img, validity))
    
    def train(self, epochs, batch_size=128, sample_interval = 50):
        (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
        X_train = (X_train.astype(np.float32)-127.5)/127.5
        X_train = np.expand_dims(X_train, axis=3)
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))
        
        for epoch in range(epochs):
            for _ in range(self.n_critic):
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)
                d_loss_real = self.critic.train_on_batch(imgs, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)
                
            g_loss = self.combined.train_on_batch(noise, valid)
            print("Epoch: %d | D loss: %f | G loss: %d"%(epoch, 1-d_loss[0], 1-g_loss[0]))
            
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
    def sample_images(self, epoch):
        r,c = 5, 5
        
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :, :, 0], 'gray')
                axs[i,j].axis('off')
                cnt += 1
                fig.savefig("w_images/mnist_%d.png"%epoch)
                plt.close()
                

In [6]:
if __name__ == "__main__":
    wgan = WGAN()
    wgan.train(epochs=4000, batch_size = 32, sample_interval = 50)

Epoch: 0 | D loss: 0.999922 | G loss: 1
Epoch: 1 | D loss: 0.999926 | G loss: 1
Epoch: 2 | D loss: 0.999929 | G loss: 1
Epoch: 3 | D loss: 0.999925 | G loss: 1
Epoch: 4 | D loss: 0.999930 | G loss: 1
Epoch: 5 | D loss: 0.999932 | G loss: 1
Epoch: 6 | D loss: 0.999937 | G loss: 1
Epoch: 7 | D loss: 0.999933 | G loss: 1
Epoch: 8 | D loss: 0.999928 | G loss: 1
Epoch: 9 | D loss: 0.999925 | G loss: 1
Epoch: 10 | D loss: 0.999932 | G loss: 1
Epoch: 11 | D loss: 0.999935 | G loss: 1
Epoch: 12 | D loss: 0.999939 | G loss: 1
Epoch: 13 | D loss: 0.999921 | G loss: 1
Epoch: 14 | D loss: 0.999938 | G loss: 1
Epoch: 15 | D loss: 0.999938 | G loss: 1
Epoch: 16 | D loss: 0.999938 | G loss: 1
Epoch: 17 | D loss: 0.999943 | G loss: 1
Epoch: 18 | D loss: 0.999940 | G loss: 1
Epoch: 19 | D loss: 0.999940 | G loss: 1
Epoch: 20 | D loss: 0.999946 | G loss: 1
Epoch: 21 | D loss: 0.999952 | G loss: 1
Epoch: 22 | D loss: 0.999951 | G loss: 1
Epoch: 23 | D loss: 0.999950 | G loss: 1
Epoch: 24 | D loss: 0.9999

Epoch: 198 | D loss: 0.999971 | G loss: 1
Epoch: 199 | D loss: 0.999971 | G loss: 1
Epoch: 200 | D loss: 0.999970 | G loss: 1
Epoch: 201 | D loss: 0.999968 | G loss: 1
Epoch: 202 | D loss: 0.999971 | G loss: 1
Epoch: 203 | D loss: 0.999965 | G loss: 1
Epoch: 204 | D loss: 0.999972 | G loss: 1
Epoch: 205 | D loss: 0.999970 | G loss: 1
Epoch: 206 | D loss: 0.999967 | G loss: 1
Epoch: 207 | D loss: 0.999971 | G loss: 1
Epoch: 208 | D loss: 0.999977 | G loss: 1
Epoch: 209 | D loss: 0.999968 | G loss: 1
Epoch: 210 | D loss: 0.999969 | G loss: 1
Epoch: 211 | D loss: 0.999982 | G loss: 1
Epoch: 212 | D loss: 0.999972 | G loss: 1
Epoch: 213 | D loss: 0.999973 | G loss: 1
Epoch: 214 | D loss: 0.999957 | G loss: 1
Epoch: 215 | D loss: 0.999971 | G loss: 1
Epoch: 216 | D loss: 0.999983 | G loss: 1
Epoch: 217 | D loss: 0.999969 | G loss: 1
Epoch: 218 | D loss: 0.999972 | G loss: 1
Epoch: 219 | D loss: 0.999964 | G loss: 1
Epoch: 220 | D loss: 0.999971 | G loss: 1
Epoch: 221 | D loss: 0.999970 | G 

Epoch: 394 | D loss: 0.999968 | G loss: 1
Epoch: 395 | D loss: 0.999973 | G loss: 1
Epoch: 396 | D loss: 0.999977 | G loss: 1
Epoch: 397 | D loss: 0.999969 | G loss: 1
Epoch: 398 | D loss: 0.999967 | G loss: 1
Epoch: 399 | D loss: 0.999980 | G loss: 1
Epoch: 400 | D loss: 0.999965 | G loss: 1
Epoch: 401 | D loss: 0.999967 | G loss: 1
Epoch: 402 | D loss: 0.999962 | G loss: 1
Epoch: 403 | D loss: 0.999963 | G loss: 1
Epoch: 404 | D loss: 0.999964 | G loss: 1
Epoch: 405 | D loss: 0.999974 | G loss: 1
Epoch: 406 | D loss: 0.999962 | G loss: 1
Epoch: 407 | D loss: 0.999969 | G loss: 1
Epoch: 408 | D loss: 0.999969 | G loss: 1
Epoch: 409 | D loss: 0.999970 | G loss: 1
Epoch: 410 | D loss: 0.999971 | G loss: 1
Epoch: 411 | D loss: 0.999970 | G loss: 1
Epoch: 412 | D loss: 0.999969 | G loss: 1
Epoch: 413 | D loss: 0.999961 | G loss: 1
Epoch: 414 | D loss: 0.999970 | G loss: 1
Epoch: 415 | D loss: 0.999971 | G loss: 1
Epoch: 416 | D loss: 0.999975 | G loss: 1
Epoch: 417 | D loss: 0.999964 | G 

Epoch: 590 | D loss: 0.999966 | G loss: 1
Epoch: 591 | D loss: 0.999974 | G loss: 1
Epoch: 592 | D loss: 0.999977 | G loss: 1
Epoch: 593 | D loss: 0.999972 | G loss: 1
Epoch: 594 | D loss: 0.999965 | G loss: 1
Epoch: 595 | D loss: 0.999978 | G loss: 1
Epoch: 596 | D loss: 0.999979 | G loss: 1
Epoch: 597 | D loss: 0.999970 | G loss: 1
Epoch: 598 | D loss: 0.999980 | G loss: 1
Epoch: 599 | D loss: 0.999971 | G loss: 1
Epoch: 600 | D loss: 0.999966 | G loss: 1
Epoch: 601 | D loss: 0.999971 | G loss: 1
Epoch: 602 | D loss: 0.999969 | G loss: 1
Epoch: 603 | D loss: 0.999975 | G loss: 1
Epoch: 604 | D loss: 0.999964 | G loss: 1
Epoch: 605 | D loss: 0.999967 | G loss: 1
Epoch: 606 | D loss: 0.999967 | G loss: 1
Epoch: 607 | D loss: 0.999974 | G loss: 1
Epoch: 608 | D loss: 0.999966 | G loss: 1
Epoch: 609 | D loss: 0.999977 | G loss: 1
Epoch: 610 | D loss: 0.999968 | G loss: 1
Epoch: 611 | D loss: 0.999970 | G loss: 1
Epoch: 612 | D loss: 0.999974 | G loss: 1
Epoch: 613 | D loss: 0.999966 | G 

Epoch: 786 | D loss: 0.999971 | G loss: 1
Epoch: 787 | D loss: 0.999967 | G loss: 1
Epoch: 788 | D loss: 0.999972 | G loss: 1
Epoch: 789 | D loss: 0.999973 | G loss: 1
Epoch: 790 | D loss: 0.999973 | G loss: 1
Epoch: 791 | D loss: 0.999961 | G loss: 1
Epoch: 792 | D loss: 0.999963 | G loss: 1
Epoch: 793 | D loss: 0.999974 | G loss: 1
Epoch: 794 | D loss: 0.999975 | G loss: 1
Epoch: 795 | D loss: 0.999971 | G loss: 1
Epoch: 796 | D loss: 0.999973 | G loss: 1
Epoch: 797 | D loss: 0.999976 | G loss: 1
Epoch: 798 | D loss: 0.999962 | G loss: 1
Epoch: 799 | D loss: 0.999972 | G loss: 1
Epoch: 800 | D loss: 0.999972 | G loss: 1
Epoch: 801 | D loss: 0.999977 | G loss: 1
Epoch: 802 | D loss: 0.999974 | G loss: 1
Epoch: 803 | D loss: 0.999978 | G loss: 1
Epoch: 804 | D loss: 0.999980 | G loss: 1
Epoch: 805 | D loss: 0.999966 | G loss: 1
Epoch: 806 | D loss: 0.999963 | G loss: 1
Epoch: 807 | D loss: 0.999972 | G loss: 1
Epoch: 808 | D loss: 0.999976 | G loss: 1
Epoch: 809 | D loss: 0.999969 | G 

...

Epoch: 3083 | D loss: 0.999970 | G loss: 1
Epoch: 3084 | D loss: 0.999958 | G loss: 1
Epoch: 3085 | D loss: 0.999959 | G loss: 1
Epoch: 3086 | D loss: 0.999967 | G loss: 1
Epoch: 3087 | D loss: 0.999969 | G loss: 1
Epoch: 3088 | D loss: 0.999977 | G loss: 1
Epoch: 3089 | D loss: 0.999971 | G loss: 1
Epoch: 3090 | D loss: 0.999982 | G loss: 1
Epoch: 3091 | D loss: 0.999978 | G loss: 1
Epoch: 3092 | D loss: 0.999969 | G loss: 1
Epoch: 3093 | D loss: 0.999975 | G loss: 1
Epoch: 3094 | D loss: 0.999964 | G loss: 1
Epoch: 3095 | D loss: 0.999981 | G loss: 1
Epoch: 3096 | D loss: 0.999985 | G loss: 1
Epoch: 3097 | D loss: 0.999967 | G loss: 1
Epoch: 3098 | D loss: 0.999968 | G loss: 1
Epoch: 3099 | D loss: 0.999972 | G loss: 1
Epoch: 3100 | D loss: 0.999968 | G loss: 1
Epoch: 3101 | D loss: 0.999966 | G loss: 1
Epoch: 3102 | D loss: 0.999976 | G loss: 1
Epoch: 3103 | D loss: 0.999972 | G loss: 1
Epoch: 3104 | D loss: 0.999967 | G loss: 1
Epoch: 3105 | D loss: 0.999977 | G loss: 1
Epoch: 3106

Epoch: 3274 | D loss: 0.999965 | G loss: 1
Epoch: 3275 | D loss: 0.999972 | G loss: 1
Epoch: 3276 | D loss: 0.999982 | G loss: 1
Epoch: 3277 | D loss: 0.999977 | G loss: 1
Epoch: 3278 | D loss: 0.999965 | G loss: 1
Epoch: 3279 | D loss: 0.999965 | G loss: 1
Epoch: 3280 | D loss: 0.999969 | G loss: 1
Epoch: 3281 | D loss: 0.999972 | G loss: 1
Epoch: 3282 | D loss: 0.999961 | G loss: 1
Epoch: 3283 | D loss: 0.999967 | G loss: 1
Epoch: 3284 | D loss: 0.999958 | G loss: 1
Epoch: 3285 | D loss: 0.999977 | G loss: 1
Epoch: 3286 | D loss: 0.999974 | G loss: 1
Epoch: 3287 | D loss: 0.999977 | G loss: 1
Epoch: 3288 | D loss: 0.999972 | G loss: 1
Epoch: 3289 | D loss: 0.999966 | G loss: 1
Epoch: 3290 | D loss: 0.999966 | G loss: 1
Epoch: 3291 | D loss: 0.999966 | G loss: 1
Epoch: 3292 | D loss: 0.999973 | G loss: 1
Epoch: 3293 | D loss: 0.999968 | G loss: 1
Epoch: 3294 | D loss: 0.999968 | G loss: 1
Epoch: 3295 | D loss: 0.999965 | G loss: 1
Epoch: 3296 | D loss: 0.999969 | G loss: 1
Epoch: 3297

Epoch: 3465 | D loss: 0.999963 | G loss: 1
Epoch: 3466 | D loss: 0.999963 | G loss: 1
Epoch: 3467 | D loss: 0.999974 | G loss: 1
Epoch: 3468 | D loss: 0.999965 | G loss: 1
Epoch: 3469 | D loss: 0.999966 | G loss: 1
Epoch: 3470 | D loss: 0.999976 | G loss: 1
Epoch: 3471 | D loss: 0.999969 | G loss: 1
Epoch: 3472 | D loss: 0.999971 | G loss: 1
Epoch: 3473 | D loss: 0.999969 | G loss: 1
Epoch: 3474 | D loss: 0.999972 | G loss: 1
Epoch: 3475 | D loss: 0.999971 | G loss: 1
Epoch: 3476 | D loss: 0.999974 | G loss: 1
Epoch: 3477 | D loss: 0.999968 | G loss: 1
Epoch: 3478 | D loss: 0.999970 | G loss: 1
Epoch: 3479 | D loss: 0.999967 | G loss: 1
Epoch: 3480 | D loss: 0.999967 | G loss: 1
Epoch: 3481 | D loss: 0.999970 | G loss: 1
Epoch: 3482 | D loss: 0.999978 | G loss: 1
Epoch: 3483 | D loss: 0.999969 | G loss: 1
Epoch: 3484 | D loss: 0.999971 | G loss: 1
Epoch: 3485 | D loss: 0.999968 | G loss: 1
Epoch: 3486 | D loss: 0.999968 | G loss: 1
Epoch: 3487 | D loss: 0.999969 | G loss: 1
Epoch: 3488

Epoch: 3656 | D loss: 0.999977 | G loss: 1
Epoch: 3657 | D loss: 0.999972 | G loss: 1
Epoch: 3658 | D loss: 0.999969 | G loss: 1
Epoch: 3659 | D loss: 0.999976 | G loss: 1
Epoch: 3660 | D loss: 0.999968 | G loss: 1
Epoch: 3661 | D loss: 0.999974 | G loss: 1
Epoch: 3662 | D loss: 0.999973 | G loss: 1
Epoch: 3663 | D loss: 0.999969 | G loss: 1
Epoch: 3664 | D loss: 0.999968 | G loss: 1
Epoch: 3665 | D loss: 0.999958 | G loss: 1
Epoch: 3666 | D loss: 0.999966 | G loss: 1
Epoch: 3667 | D loss: 0.999974 | G loss: 1
Epoch: 3668 | D loss: 0.999976 | G loss: 1
Epoch: 3669 | D loss: 0.999970 | G loss: 1
Epoch: 3670 | D loss: 0.999968 | G loss: 1
Epoch: 3671 | D loss: 0.999973 | G loss: 1
Epoch: 3672 | D loss: 0.999968 | G loss: 1
Epoch: 3673 | D loss: 0.999973 | G loss: 1
Epoch: 3674 | D loss: 0.999965 | G loss: 1
Epoch: 3675 | D loss: 0.999975 | G loss: 1
Epoch: 3676 | D loss: 0.999975 | G loss: 1
Epoch: 3677 | D loss: 0.999967 | G loss: 1
Epoch: 3678 | D loss: 0.999968 | G loss: 1
Epoch: 3679

Epoch: 3847 | D loss: 0.999968 | G loss: 1
Epoch: 3848 | D loss: 0.999958 | G loss: 1
Epoch: 3849 | D loss: 0.999975 | G loss: 1
Epoch: 3850 | D loss: 0.999973 | G loss: 1
Epoch: 3851 | D loss: 0.999967 | G loss: 1
Epoch: 3852 | D loss: 0.999970 | G loss: 1
Epoch: 3853 | D loss: 0.999969 | G loss: 1
Epoch: 3854 | D loss: 0.999965 | G loss: 1
Epoch: 3855 | D loss: 0.999979 | G loss: 1
Epoch: 3856 | D loss: 0.999968 | G loss: 1
Epoch: 3857 | D loss: 0.999969 | G loss: 1
Epoch: 3858 | D loss: 0.999966 | G loss: 1
Epoch: 3859 | D loss: 0.999968 | G loss: 1
Epoch: 3860 | D loss: 0.999969 | G loss: 1
Epoch: 3861 | D loss: 0.999976 | G loss: 1
Epoch: 3862 | D loss: 0.999956 | G loss: 1
Epoch: 3863 | D loss: 0.999974 | G loss: 1
Epoch: 3864 | D loss: 0.999967 | G loss: 1
Epoch: 3865 | D loss: 0.999973 | G loss: 1
Epoch: 3866 | D loss: 0.999972 | G loss: 1
Epoch: 3867 | D loss: 0.999969 | G loss: 1
Epoch: 3868 | D loss: 0.999963 | G loss: 1
Epoch: 3869 | D loss: 0.999972 | G loss: 1
Epoch: 3870